In [1]:
from bs4 import BeautifulSoup

#pulling note names and durations from the .xml file
f = open('tell.xml', encoding='utf-8')
xml = BeautifulSoup(f.read(), 'html.parser')
f.close()

#this dictionary isn't actually used, but it's nice to have sometimes
part_names = {instrument.attrs['id']:instrument.find('instrument-name').get_text() for instrument in xml.find_all('score-part')}

parts = {}
names = ['A', 'B', 'C', 'D', 'E', 'F', 'G'] # the basic 7 note names without accidentals
tie = False #this variable keeps track of whether a tie is in effect or not, being switched inside of the loop

for part in xml.find_all('part'):
    notes = {'pitches':[], 'durations':[]}
    for note in part.find_all('note'):
        if note.find('voice').get_text() != '1': #all voices other than the leading voice are ignored. if the leading voice splits, then notes will be output from bottom to top, which causes playback issues
            continue
        if note.find('rest') != None: #if the note is a rest, this branch executes, which sets pitch to zero and skips to the duration section
            notes['pitches'].append(0)
        elif tie:
            notes['durations'][-1] += int(note.find('duration').get_text()) #if a tie is in effect then the duration of the note is extended, rather than creating a new note
            if note.find('tie') != None and note.find('tie').attrs['type'] == 'stop': #if the tie has ended then exit out of tie mode
                tie = False
            continue #skip the duration appending at the end of the loop
        else:
            if note.find('tie') != None: #if the tie element appears then assume the beginning of a tie
                tie = True
            step = note.find('step').get_text()
            octave = note.find('octave').get_text()
            alter = note.find('alter')
            if alter != None:
                accidental = alter.get_text()
                if accidental == '-1':
                    if step == 'C' or step == 'F': # Cb is the enharmonic equivalent of B, while Fb is the enharmonic equivalent of E
                        step = names[names.index(step) - 1]
                    else:
                        step = step + 'b'
                else:
                    if step == 'B' or step == 'E': # B# is the enharmonic equivalent of C, while E# is the enharmonic equivalent of F
                        step = names[names.index(step) + 1]
                    else:
                        step = step + '#'
            notes['pitches'].append(step + octave)
        notes['durations'].append(int(note.find('duration').get_text()))
    
    parts[part.attrs['id']] = notes
    
parts

{'P1': {'pitches': ['B4',
   'B4',
   'B4',
   'B4',
   'B4',
   'B4',
   'B4',
   'G#4',
   'E4',
   'G#4',
   'B4',
   'G#4',
   'B4',
   'E5',
   'B4',
   'G#4',
   'E4',
   'G#4',
   'B4',
   'G#4',
   'B4',
   'E5',
   'B4',
   'B4',
   'B4',
   'B4',
   'B4',
   'B4',
   'B4',
   'B4',
   'B4',
   'B4',
   'B4',
   'B4',
   'B4',
   'B4',
   'B4',
   'B4',
   'B4',
   'B4',
   'B4',
   'B4',
   'B4',
   'B4',
   'B4',
   'B4',
   'B4',
   'B4',
   'B4',
   'B4',
   'B4',
   'B4',
   'B4',
   'B4',
   0,
   0,
   0,
   'B3',
   'B3',
   'B3',
   'B3',
   'B3',
   'B3',
   'B3',
   'B3',
   'E4',
   'F#4',
   'G#4',
   'B3',
   'B3',
   'B3',
   'B3',
   'B3',
   'E4',
   'G#4',
   'G#4',
   'F#4',
   'D#4',
   'B3',
   'B3',
   'B3',
   'B3',
   'B3',
   'B3',
   'B3',
   'B3',
   'B3',
   'E4',
   'F#4',
   'G#4',
   'E4',
   'G#4',
   'B4',
   'A4',
   'G#4',
   'F#4',
   'E4',
   'G#4',
   'E4',
   'B4',
   'B4',
   'B4',
   'B4',
   'B4',
   'B4',
   'B4',
   'B4',
   'E5',
  

In [2]:
#converting to arduino friendly list
ino_names = []

for note in parts['P1']['pitches']:
    if note == 0:
        ino_names.append(str(note)) #zeros have to be typecast to strings so that i can print them
        continue
    if len(note) == 2: #this evaluates to true when there are no accidentals
        ino_names.append('NOTE_' + note)
    else:
        name, accidental = note[0], note[1]
        if accidental == 'b': #the defined pitches only include sharps and no flats, so flat notes have to be converted to their enharmonic equivalents
            name = names[names.index(name) - 1]
        ino_names.append('NOTE_' + name + 'S' + note[-1])

print(', '.join(ino_names[:428])) #the piece is truncated at 428 notes because it takes up too much memory on the arduino otherwise
print(', '.join(map(lambda num:str(num), parts['P1']['durations'][:428])))

NOTE_B4, NOTE_B4, NOTE_B4, NOTE_B4, NOTE_B4, NOTE_B4, NOTE_B4, NOTE_GS4, NOTE_E4, NOTE_GS4, NOTE_B4, NOTE_GS4, NOTE_B4, NOTE_E5, NOTE_B4, NOTE_GS4, NOTE_E4, NOTE_GS4, NOTE_B4, NOTE_GS4, NOTE_B4, NOTE_E5, NOTE_B4, NOTE_B4, NOTE_B4, NOTE_B4, NOTE_B4, NOTE_B4, NOTE_B4, NOTE_B4, NOTE_B4, NOTE_B4, NOTE_B4, NOTE_B4, NOTE_B4, NOTE_B4, NOTE_B4, NOTE_B4, NOTE_B4, NOTE_B4, NOTE_B4, NOTE_B4, NOTE_B4, NOTE_B4, NOTE_B4, NOTE_B4, NOTE_B4, NOTE_B4, NOTE_B4, NOTE_B4, NOTE_B4, NOTE_B4, NOTE_B4, NOTE_B4, 0, 0, 0, NOTE_B3, NOTE_B3, NOTE_B3, NOTE_B3, NOTE_B3, NOTE_B3, NOTE_B3, NOTE_B3, NOTE_E4, NOTE_FS4, NOTE_GS4, NOTE_B3, NOTE_B3, NOTE_B3, NOTE_B3, NOTE_B3, NOTE_E4, NOTE_GS4, NOTE_GS4, NOTE_FS4, NOTE_DS4, NOTE_B3, NOTE_B3, NOTE_B3, NOTE_B3, NOTE_B3, NOTE_B3, NOTE_B3, NOTE_B3, NOTE_B3, NOTE_E4, NOTE_FS4, NOTE_GS4, NOTE_E4, NOTE_GS4, NOTE_B4, NOTE_A4, NOTE_GS4, NOTE_FS4, NOTE_E4, NOTE_GS4, NOTE_E4, NOTE_B4, NOTE_B4, NOTE_B4, NOTE_B4, NOTE_B4, NOTE_B4, NOTE_B4, NOTE_B4, NOTE_E5, NOTE_FS5, NOTE_GS5, NOTE_B4,

In [3]:
len(ino_names)

1273